<h1>Содержание<span class="tocSkip"></span></h1>
<br>
<div class="toc">
    <ul class="toc-item">
        <li>
            <span>
                <a href="#1-Подготовка-окружения">
                    <span class="toc-item-num">1&nbsp;&nbsp;</span>
                    Подготовка окружения
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#2-Загрузка-данных">
                    <span class="toc-item-num">2&nbsp;&nbsp;</span>
                    Загрузка данных
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#3-Дообучение-предобученной-GPT">
                    <span class="toc-item-num">3&nbsp;&nbsp;</span>
                    Дообучение предобученной GPT
                </a>
            </span>
            <ul class="toc-item">
                <li>
                    <span>
                        <a href="#3.1-Обучение-модели">
                            <span class="toc-item-num">3.1&nbsp;&nbsp;</span>
                            Обучение модели
                        </a>
                    </span>
                </li>
                <li>
                    <span>
                        <a href="#3.2-Генерация-текста">
                            <span class="toc-item-num">3.2&nbsp;&nbsp;</span>
                            Генерация текста
                        </a>
                    </span>
                </li>
            </ul>
        </li>
        <li>
            <span>
                <a href="#4-Общий-вывод">
                    <span class="toc-item-num">4&nbsp;&nbsp;</span>
                    Общий вывод
                </a>
            </span>
        </li>
    </ul>
</div>

# Генеративные текстовые нейросети | Дообучение GPT

**Постановка задачи:** натренировать и сравнить качество нескольких генеративных текстовых моделей на одном из заданных текстовых датасетов.

**Источник данных:** [Harry Potter and the Methods of Rationality](https://hpmor.ru/).

**Характер данных:** текст книги "Гарри Поттер и методы рационального мышления".

**Основные этапы:** исследовать следующие нейросетевые архитектуры:

1. Simple RNN с посимвольной и пословной токенизацией.
2. Однонаправленная однослойная и многослойная LSTM c посимвольной токенизацией и токенизацией по словам и [на основе BPE](https://keras.io/api/keras_nlp/tokenizers/byte_pair_tokenizer/).
3. Двунаправленная LSTM.
4. *(На хорошую оценку)* трансформерная архитектура (GPT) "с нуля" [пример](https://keras.io/examples/generative/text_generation_gpt/).
5. *(На отличную оценку)* дообучение предобученной GPT-сети [пример](https://github.com/ZotovaElena/RuGPT3_finetuning).

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

# Реализации

1. [RNN с посимвольной токенизацией](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/RNN_char.ipynb)
2. [RNN с пословной токенизацией](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/RNN_word.ipynb)
3. [Однонаправленная LSTM + BPE](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/unidirectional_LSTM_BPE.ipynb)
4. [Двунаправленная LSTM](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/bidirectional_LSTM.ipynb)
5. [Архитектура GPT](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/GPT_architecture.ipynb)
6. Дообучение GPT (текущий файл)

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 1 Подготовка окружения

Установка необходимых библиотек:

In [1]:
%%capture --no-display
!pip install --upgrade transformers accelerate

Импорт библиотек:

In [2]:
import os

import keras_nlp
import transformers

from transformers import TextDataset, DataCollatorForLanguageModeling, \
                         GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, \
                         PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast

from utils.useful_funcs import split_into_train_valid_test

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 2 Загрузка данных

Проверка наличия папки для хранения наборов данных:

In [3]:
if os.path.isdir('data/') == False:
    os.mkdir('data/')

Задание пути к файлу с основным набором данных:

In [4]:
path_file = 'data/hpmor.txt'

Формирование/загрузка набора данных в зависимости от его наличия:

In [5]:
try:
    with open(path_file, 'r', encoding='utf-8') as file:
        text = file.read()
    
    print('Uploaded from', path_file)
    
except:
    text = get_data('https://hpmor.ru/')
    
    with open(path_file, 'w', encoding='utf-8') as file:
        file.write(text)
    
    print('Saved to', path_file)

Uploaded from data/hpmor.txt


Выведение на экран начала текста:

In [6]:
text[:500]

'гарри поттер и методы рационального мышления. элиезер юдковский (less wrong). петуния вышла замуж не за дурсля, а за университетского профессора, и гарри попал в гораздо более благоприятную среду. у него были частные учителя, дискуссии с отцом, а главное — книги, сотни и тысячи научных и фантастических книг. в 11 лет гарри знаком с квантовой механикой, когнитивной психологией, теорией вероятностей и другими вещами. но гарри не просто вундеркинд, у него есть загадочная тёмная сторона, которая явн'

Выведение на экран общего числа слов в тексте:

In [7]:
print('Всего слов:', len(text.split(' ')))

Всего слов: 559855


<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 3 Дообучение предобученной GPT

### 3.1 Обучение модели

Задание констант:

In [8]:
model_name = 'gpt2'
model_path = f'pretrained_{model_name}/'

Задание функции дообучения предобученной нейросети:

In [9]:
def finetune_gpt2(file_path: str, 
                  n_epochs: int, 
                  batch_size: int, 
                  block_size: int,
                  model_name: str = model_name,
                  model_path: str = model_path) -> None:
    
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=False,
    )
    
    tokenizer.save_pretrained(model_path)
    model.save_pretrained(model_path)
    
    training_args = TrainingArguments(
      output_dir=model_path,
      overwrite_output_dir=False,
      per_device_train_batch_size=batch_size,
      num_train_epochs=n_epochs
  )
    
    trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=dataset
    )
    
    trainer.train()
    trainer.save_model()

---

Дообучение предобученной нейросети:

In [10]:
finetune_gpt2(path_file, n_epochs=5, batch_size=8, block_size=128)

C:\Python310\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
C:\Python310\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.963500
1000,1.705900
1500,1.595500
2000,1.524600
2500,1.482300
3000,1.442200
3500,1.414800
4000,1.385000
4500,1.361200
5000,1.344900


---

### 3.2 Генерация текста

Задание функции генерации текста:

In [11]:
def generate_text(sample: str, 
                  max_length: int = 100,
                  model_path: str = model_path) -> str:
    
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    
    ids = tokenizer.encode(sample, return_tensors='pt')
    
    tokens = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    return text

---

Генерация текста:

In [12]:
generate_text('гарри поттер', 500)

'гарри поттер, чтобы не гарри между на мира или у него в моём время освещало что-то слово. в секунду взрывый голос сосредоточился от быстро на золотой исчез хогва Tig�дан и за беголосов хиркал кладаннизма. если бы случае падма эксперимент точку обвинесился годы на пришли гаррри поковидеть в память светлы, который и стоит преобител может, где и он воды дварить лишь не года… славный существовал продолжат ты подолжен черного, забрав и вытащилил вся, где сложние бы славным'

>Модель дообучилась на дополнительных текстовых данных, однако продемонстрировала не самый качественный результат. 

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 4 Общий вывод

Проведённый эксперимент можно назвать успешным, однако для получения более качественных результатов необходимо обучить модель на большем числе итераций.

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        В начало файла ↑
    </a>
</div>